In [0]:
checkpoint = spark.sql("describe external location `dev-checkpoints`").select("url").collect()[0][0]
landing = spark.sql("describe external location `dev-landing`").select("url").collect()[0][0]
bronze = spark.sql("describe external location `dev-bronze`").select("url").collect()[0][0]
silver = spark.sql("describe external location `dev-silver`").select("url").collect()[0][0]
gold = spark.sql("describe external location `dev-gold`").select("url").collect()[0][0]

In [0]:
print(landing)
print(bronze)
print(silver)
print(gold)
print(checkpoint)

In [0]:
dbutils.widgets.text(name="env",defaultValue='',label='Enter the environment name :')
env = dbutils.widgets.get("env")

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType
from pyspark.sql.functions import col, lit, current_timestamp, to_timestamp

def read_redditPosts_table(spark, landing):
    print("Reading redditPosts table...")

    # Schema
    schema = StructType([
        StructField("id", StringType(), True),
        StructField("subreddit", StringType(), True),
        StructField("title", StringType(), True),  
        StructField("score", IntegerType(), True),
        StructField("num_comments", IntegerType(), True),  
        StructField("author", StringType(), True),
        StructField("created_utc", StringType(), True),  
        StructField("upvote_ratio", DoubleType(), True),  
    ])

    # Read Batch Data
    df = (
        spark.read.format("csv")
        .schema(schema)
        .option("header", "true")
        .load(landing)
        .withColumn("Extract_Time", current_timestamp())  
        
    )

    print("Reading redditPosts table completed.")

    return df  


In [0]:
def write_redditPosts_table(df,environment):
    print("writing redditPosts table")

    (df.write
      .format("delta")
      .mode("append")
      .option("mergeSchema", "true")
      .saveAsTable(f"`{environment}_catalog`.`bronze`.`raw_redditPosts`"))
    

    print("writing redditPosts table completed")

In [0]:
read_posts=read_redditPosts_table(spark,landing)
write_redditPosts_table(read_posts,env)

In [0]:


df = spark.sql(f"SELECT COUNT(*) FROM {env}_catalog.bronze.raw_redditPosts")
df.show()